In [1]:
import requests
import lxml.html as lh
import pandas as pd
import numpy as np

In [2]:
# Now we assign the link of the website through which we are going to scrape the data
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

# Create a handle, page, to handle the contents of the website
page = requests.get(url)

# Store the contents of the website under doc
doc = lh.fromstring(page.content)

# Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')

In [3]:
# Create empty list col and a counter index i which with initial value 0
col=[]
i=0

# For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print ('%d: %s' % (i,name))
    col.append((name,[]))

1: Postal code

2: Borough

3: Neighborhood



In [4]:
# For sanity check, ensure that all the rows have the same width. If not, we probably got something more than just the table.
#Check the length of the first 12 rows
[len(T) for T in tr_elements[:12]]

[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

In [5]:
# Creating Pandas DataFrame - each header is appended to a tuple along with an empty list.
# Since out first row is the header, data is stored on the second row onwards
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 3, the //tr data is not from our table 
    if len(T)!=3:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

In [6]:
# Just to be sure, let’s check the length of each column. Ideally, they should all be the same.
[len(C) for (title,C) in col]

[181, 181, 181]

In [7]:
# Now we are ready to create the DataFrame:
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)

# The column names and the values now have \n at the end, let's remove it:
df.replace(regex=r'\n', value='',inplace=True)
df.rename(columns={"Postal code\n":"Postal code","Borough\n":"Borough","Neighborhood\n":"Neighborhood"},inplace=True)

# When there are multiple values in Neighborhood  column, they are separated by "/", let's use "," instead:
df.replace(regex=r' /', value=',',inplace=True)

In [8]:
# Remove all the rows where Borough has a "Not assigned" value
df_cleaned = df[df.Borough !="Not assigned"]

# The last row of the table does not have observations anymore, so let's remove it as well
df_final = df_cleaned[df_cleaned.Borough !="Canadian postal codes"]

In [9]:
# Reset the index counter to start from 0 again and omit values that were removed
df_final.reset_index(inplace=True, drop=True)

# Show the final version of the dataframe after all the amendments
df_final

,Postal code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road , Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,Business reply mail Processing CentrE
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [10]:
# Use the .shape method to print the number of rows of your dataframe
df_final.shape[0]

103

In [17]:
# Import additional libraries to handle coordinates
import geocoder # import geocoder

In [18]:
col_one_list = df_final['Postal code'].tolist()

In [ ]:
import pgeocode
nomi = pgeocode.Nominatim('ca')
df_output = nomi.query_postal_code(col_one_list)



In [32]:
df_tomerge = df_output[["postal_code","latitude","longitude"]]

In [ ]:
df_merged = df_final.merge(df_tomerge, left_on="Postal code", right_on="postal_code") 
df_merged.head()

In [ ]:
# Omit the column postal_code to avoid duplication
df_merged = df_merged.drop(["postal_code"], axis=1)
df_merged.head()